<a href="https://colab.research.google.com/github/sjh7535/sjh7535/blob/main/one_hot_encoding(test)_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 라이브러리 저장
# !pip freeze > requirements.txt

In [3]:
# 라이브러리 실행
# !pip install -r requirements.txt
!pip install tensorflow==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 1.7 MB 50.1 MB/s 
     |████████████████████████████████| 5.9 MB 44.4 MB/s 
     |████████████████████████████████| 438 kB 56.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

In [4]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
train = pd.read_excel('mental_health_medical_data.xlsx')
test = pd.read_excel('mental_health_medical_data_test.xlsx')

In [6]:
def listing(x):
    x = eval(x)
    return x
# 문자열 벗기기\n"
train['symptoms'] = train['symptoms'].apply(listing)
test['symptoms'] = test['symptoms'].apply(listing)

In [7]:
# 테스트 데이터 랜덤 샘플링.
test

,Unnamed: 0,disease,symptoms,department
0,0.0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,8.0,신경성 폭식증,"[체중증가, 복부 통증, 폭식, 구토]",정신건강의학과
3,NaN,NaN,"[감각 과민, 대인관계 어려움, 감정 변화]",NaN
4,NaN,NaN,"[반사회적 인격장애, 금단증상]",NaN


In [8]:
train

,Unnamed: 0,disease,symptoms,department
0,0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,2,공황 장애,"[숨막히는 느낌, 얼굴이 화끈거림, 가슴 통증, 죽음에 대한 공포, 어지러움, 발한...",정신건강의학과
3,3,금단 현상,"[복시, 손떨림, 고혈압, 서맥, 저혈압, 청색증, 불안, 기운없음, 불면증, 두통...",정신건강의학과
4,4,분열정동 장애,"[와해된 언어, 피해망상, 환각, 환시, 환청, 와해된 행동, 망상, 이상행동]",정신건강의학과
...,...,...,...,...
59,59,혈관성 치매,"[언어장애, 인지장애, 감각 이상, 마비]",정신건강의학과
60,60,히스테리성 인격장애,[],정신건강의학과
61,61,레트 증후군,"[상동적 행동, 인지장애, 운동장애, 정신 지체]",정신건강의학과
62,62,폐소공포증,"[불안, 좁은 공간에서의 불안감]",정신건강의학과


In [9]:
#레이블 인코딩(타겟 문자열 불가)
from sklearn.preprocessing import LabelEncoder
items = train['disease']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

print(items)
print('인코딩 변환값:',labels)
print(len(labels))

0          강박 장애
1       경계성 인격장애
2          공황 장애
3          금단 현상
4        분열정동 장애
         ...    
59        혈관성 치매
60    히스테리성 인격장애
61        레트 증후군
62         폐소공포증
63       과호흡 증후군
Name: disease, Length: 64, dtype: object
인코딩 변환값: [ 0  2  3  6 22 23 24 35 36 37 39 41 43 47 51 52 59  1  5  7  8  9 10 12
 13 14 15 16 17 18 19 20 21 25 26 27 28 29 30 31 32 33 34 38 40 42 44 45
 46 48 49 50 53 54 55 56 57 60 61 62 63 11 58  4]
64


In [10]:
train

,Unnamed: 0,disease,symptoms,department
0,0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,2,공황 장애,"[숨막히는 느낌, 얼굴이 화끈거림, 가슴 통증, 죽음에 대한 공포, 어지러움, 발한...",정신건강의학과
3,3,금단 현상,"[복시, 손떨림, 고혈압, 서맥, 저혈압, 청색증, 불안, 기운없음, 불면증, 두통...",정신건강의학과
4,4,분열정동 장애,"[와해된 언어, 피해망상, 환각, 환시, 환청, 와해된 행동, 망상, 이상행동]",정신건강의학과
...,...,...,...,...
59,59,혈관성 치매,"[언어장애, 인지장애, 감각 이상, 마비]",정신건강의학과
60,60,히스테리성 인격장애,[],정신건강의학과
61,61,레트 증후군,"[상동적 행동, 인지장애, 운동장애, 정신 지체]",정신건강의학과
62,62,폐소공포증,"[불안, 좁은 공간에서의 불안감]",정신건강의학과


In [11]:
X_train = np.array([x for x in train['symptoms']])
y_train = np.array(labels)

X_test = np.array([x for x in test['symptoms']])

In [12]:
X_test

array([list(['불안', '반복적인 동작', '의심']),
       list(['의심', '약물남용', '대인관계 어려움', '조절할 수 없는 충동감', '자살의 위험', '우울', '망상']),
       list(['체중증가', '복부 통증', '폭식', '구토']),
       list(['감각 과민', '대인관계 어려움', '감정 변화']), list(['반사회적 인격장애', '금단증상'])],
      dtype=object)

In [13]:
X_train[0]

['불안', '반복적인 동작', '의심']

# **모델링**

In [14]:
#파라미터 설정
vocab_size = 178
embedding_dim = 8 # 임베딩 차원
max_length = 17      # 증상 최대 개수 
padding_type='post'

In [15]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index


In [16]:
word_index

{'불안': 1,
 '우울': 2,
 '언어장애': 3,
 '대인관계 어려움': 4,
 '망상': 5,
 '감정 변화': 6,
 '수면장애': 7,
 '가슴 두근거림': 8,
 '불면증': 9,
 '환각': 10,
 '산만함': 11,
 '기억장애': 12,
 '어지러움': 13,
 '손떨림': 14,
 '초조함': 15,
 '인지장애': 16,
 '의심': 17,
 '조절할 수 없는 충동감': 18,
 '얼굴이 화끈거림': 19,
 '감각 이상': 20,
 '체중감소': 21,
 '구토': 22,
 '낮은 자존감': 23,
 '의식 변화': 24,
 '긴장': 25,
 '혼돈': 26,
 '충동 조절 어려움': 27,
 '정신 지체': 28,
 '약물남용': 29,
 '자살의 위험': 30,
 '발한': 31,
 '가슴 답답': 32,
 '오심': 33,
 '온몸이 떨림': 34,
 '저혈압': 35,
 '두통': 36,
 '환청': 37,
 '와해된 언어': 38,
 '피해망상': 39,
 '환시': 40,
 '와해된 행동': 41,
 '이상행동': 42,
 '식욕부진': 43,
 '피로감': 44,
 '무월경': 45,
 '복부 통증': 46,
 '폭식': 47,
 '권태감': 48,
 '복부팽만감': 49,
 '저림': 50,
 '비현실감': 51,
 '피부 긴장도 저하': 52,
 '이상한 소리를 냄': 53,
 '비정상적인 움직임': 54,
 '낮은 지능': 55,
 '행동장애': 56,
 '지남력 장애': 57,
 '섬망': 58,
 '대화에 불쑥 끼어듬': 59,
 '인내심이 부족함': 60,
 '건망증': 61,
 '몽유병': 62,
 '수면 무호흡': 63,
 '성격의변화': 64,
 '낮은 학업 성취': 65,
 '상동적 행동': 66,
 '반복적인 동작': 67,
 '숨막히는 느낌': 68,
 '가슴 통증': 69,
 '죽음에 대한 공포': 70,
 '오한': 71,
 '복시': 72,
 '고혈압': 73,
 '서맥': 74,
 '청색증'

In [17]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [18]:
test_padded

array([[ 1, 67, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [17, 29,  4, 18, 30,  2,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [85, 46, 47, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [90,  4,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [91, 92,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]], dtype=int32)

In [19]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(64, activation='softmax')
])

In [20]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 8)             1424      
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                216       
                                                                 
 dense_1 (Dense)             (None, 64)                1600      
                                                                 
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=1000, verbose=2)
# history = model.fit(train_padded, y_train, 
#                     epochs=num_epochs, verbose=2, 
#                     validation_split=0.2)

Epoch 1/1000
2/2 - 1s - loss: 4.1597 - accuracy: 0.0156 - 1s/epoch - 509ms/step
Epoch 2/1000
2/2 - 0s - loss: 4.1588 - accuracy: 0.0156 - 9ms/epoch - 4ms/step
Epoch 3/1000
2/2 - 0s - loss: 4.1584 - accuracy: 0.0156 - 8ms/epoch - 4ms/step
Epoch 4/1000
2/2 - 0s - loss: 4.1581 - accuracy: 0.0156 - 11ms/epoch - 5ms/step
Epoch 5/1000
2/2 - 0s - loss: 4.1578 - accuracy: 0.0156 - 7ms/epoch - 4ms/step
Epoch 6/1000
2/2 - 0s - loss: 4.1575 - accuracy: 0.0156 - 9ms/epoch - 4ms/step
Epoch 7/1000
2/2 - 0s - loss: 4.1573 - accuracy: 0.0156 - 9ms/epoch - 4ms/step
Epoch 8/1000
2/2 - 0s - loss: 4.1570 - accuracy: 0.0156 - 9ms/epoch - 5ms/step
Epoch 9/1000
2/2 - 0s - loss: 4.1567 - accuracy: 0.0156 - 9ms/epoch - 4ms/step
Epoch 10/1000
2/2 - 0s - loss: 4.1563 - accuracy: 0.0156 - 9ms/epoch - 5ms/step
Epoch 11/1000
2/2 - 0s - loss: 4.1560 - accuracy: 0.0156 - 11ms/epoch - 6ms/step
Epoch 12/1000
2/2 - 0s - loss: 4.1557 - accuracy: 0.0156 - 9ms/epoch - 4ms/step
Epoch 13/1000
2/2 - 0s - loss: 4.1554 - accura

In [22]:
# 여기 부분의 train_padded 부분을 새로운 데이터 셋을 생성해주면 됨
y_prob = model.predict(test_padded, verbose=2) 
predicted = y_prob.argmax(axis=-1)
# 그리고 n번의 질문 역시도 이것을 바탕으로 진행됨

# 모델이 여러번 사용될 예정

print("학습용으로 다시 예측해봄")

encoder.classes_[predicted]

1/1 - 0s - 318ms/epoch - 318ms/step
학습용으로 다시 예측해봄


array(['강박 장애', '경계성 인격장애', '신경성 폭식증', '아스퍼거 증후군', '알코올 의존성'],
      dtype=object)

In [23]:
test_padded

array([[ 1, 67, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [17, 29,  4, 18, 30,  2,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [85, 46, 47, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [90,  4,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0],
       [91, 92,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]], dtype=int32)

In [24]:
test

,Unnamed: 0,disease,symptoms,department
0,0.0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,8.0,신경성 폭식증,"[체중증가, 복부 통증, 폭식, 구토]",정신건강의학과
3,NaN,NaN,"[감각 과민, 대인관계 어려움, 감정 변화]",NaN
4,NaN,NaN,"[반사회적 인격장애, 금단증상]",NaN


In [25]:
# 질병 라벨링 순서
print(encoder.classes_)

['강박 장애' '건강염려증' '경계성 인격장애' '공황 장애' '과호흡 증후군' '광장공포증' '금단 현상' '기분부전증'
 '난독증' '뇌경색' '뚜렛 증후군' '레트 증후군' '말더듬이' '망상 장애' '뮌하우젠 증후군' '반사회성 인격장애'
 '반응성 애착장애' '발모벽' '베르니케 코시코프 증후군' '병적 도벽' '병적 방화' '분리 불안 장애' '분열정동 장애'
 '불면증' '불안 장애' '사회공포증' '산후 우울증' '섬망' '섭식 장애' '성도착증' '성인 주의력 결핍 과잉행동장애'
 '소아청소년기 우울증' '수면 무호흡증' '수면 장애' '수면보행증' '신경성 식욕부진증' '신경성 폭식증' '신체형 장애'
 '실어증' '아스퍼거 증후군' '알츠하이머병' '알코올 의존성' '알코올성 치매' '양극성 장애' '얼렌 증후군'
 '외상 후 스트레스 장애' '외상성 치매' '우울증' '의상도착증' '인터넷 중독' '자폐증 / 자폐 스펙트럼 장애' '적응 장애'
 '조현병' '주의력결핍 과잉행동장애' '지적장애' '치매' '틱 장애' '편집성 인격장애' '폐소공포증' '하지불안증후군'
 '해리 장애' '행동 및 충동 장애' '혈관성 치매' '히스테리성 인격장애']


---
# 여기서부터 추가 



In [26]:
#증상 목록 출력
for i in word_index.keys():
  print(i)

#증상 입력받기
pat_word_index = []

pat_symp = input("\n위 증상중 해당하는 증상을 하나씩 입력해주세요 : ")
while pat_symp != '':
  pat_word_index.append(pat_symp)
  pat_symp = input("\n위 증상중 해당하는 증상을 하나씩 입력해주세요 : ")

불안
우울
언어장애
대인관계 어려움
망상
감정 변화
수면장애
가슴 두근거림
불면증
환각
산만함
기억장애
어지러움
손떨림
초조함
인지장애
의심
조절할 수 없는 충동감
얼굴이 화끈거림
감각 이상
체중감소
구토
낮은 자존감
의식 변화
긴장
혼돈
충동 조절 어려움
정신 지체
약물남용
자살의 위험
발한
가슴 답답
오심
온몸이 떨림
저혈압
두통
환청
와해된 언어
피해망상
환시
와해된 행동
이상행동
식욕부진
피로감
무월경
복부 통증
폭식
권태감
복부팽만감
저림
비현실감
피부 긴장도 저하
이상한 소리를 냄
비정상적인 움직임
낮은 지능
행동장애
지남력 장애
섬망
대화에 불쑥 끼어듬
인내심이 부족함
건망증
몽유병
수면 무호흡
성격의변화
낮은 학업 성취
상동적 행동
반복적인 동작
숨막히는 느낌
가슴 통증
죽음에 대한 공포
오한
복시
고혈압
서맥
청색증
기운없음
수면 후에도 피로함
거칠어진 피부
빈호흡
빈뇨
복부 불편감
목의 이물감
근육긴장
설사
체중증가
생리불순
구강건조
근육통
가슴 쓰림
감각 과민
반사회적 인격장애
금단증상
말을 많이 함
높아진 자존심
사고의 비약
삶에 대한 흥미 상실
오후 우울감 완화
희망이 없는 느낌
걸핏하면 눈물지음
무욕증
정서적 둔마
무논리증
다리 통증
병원 쇼핑
질병에 대한 공포
우유부단함
에너지가 적음
비관주의
읽기
쓰기 능력 부족
반신마비
시야장애
말더듬
고정된 믿음
거짓 증상 호소
직무 태만
나쁜 품행
야만적 행위
사회성 부족
저체중
말초 신경장애
보행이상
안구진탕
신경과민
충동적 도벽
흥분
애착 상대와 분리 거부
왜곡된 신체상
자제력 상실 두려움
음식섭취 거부
피학증
가학증
소아기호증
노출증
비정상적 성적 기호
접촉도착증
관음증
발기부전
입수면기의 환각
수면 마비
수면 중 이상한 행동
수면 주기의 변화
수면 시작이 어려움
낮 시간대의 졸음
수면 유지가 어려움
수면 중 보행
깨어나면 기억하지 못함
실인증
방향감각 상실
시력 감소
실행증
판단력장애
조급증
읽기 자체의 문제
읽으려 하지 않음
눈의 피로
광과민성
눈의 충혈
변실금
이성의 옷을 착용함
강

In [27]:
pat_word_array = np.array(pd.Series([pat_word_index]))

In [28]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

In [29]:
pat_padded

array([[  1, 172, 177,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]], dtype=int32)

In [31]:
#transpose 하고 행렬곱
mm_result = tf.matmul(train_padded, pd.DataFrame(pat_padded).transpose())
mm_result

<tf.Tensor: shape=(64, 1), dtype=int32, numpy=
array([[14534],
       [ 5713],
       [15549],
       [15401],
       [ 8516],
       [20876],
       [27756],
       [10976],
       [16316],
       [20932],
       [ 1840],
       [16547],
       [16615],
       [ 9173],
       [17480],
       [ 8861],
       [10157],
       [36524],
       [  176],
       [13266],
       [19029],
       [20451],
       [ 9341],
       [  113],
       [19613],
       [  115],
       [40690],
       [ 9918],
       [ 9657],
       [14064],
       [43480],
       [21697],
       [ 3632],
       [  176],
       [ 2759],
       [ 7622],
       [26313],
       [46376],
       [14410],
       [ 1573],
       [12491],
       [49176],
       [25430],
       [    3],
       [52326],
       [    0],
       [54426],
       [ 1415],
       [ 2611],
       [  160],
       [13401],
       [32823],
       [39059],
       [14731],
       [ 5130],
       [ 9341],
       [57787],
       [11064],
       [38993],
       [ 

In [36]:
max(mm_result).value_index()

AttributeError: ignored

In [33]:
#위의 mm_result에서 최댓값은
#max(mm_result)
#이며 그 인덱스는
#np.array(mm_result).tolist().index(max(mm_result))

# 정렬을 하는게 같은 값 비교도 편하고 적합성 판단에도 유리할듯

번호에 값 연결

In [34]:
sorted_arr = []

#np.array(mm_result).tolist().index(max(mm_result)) : mm_result에서 최댓값 index 반환

for i in range(len(mm_result)):
  sorted_arr.append([np.array(mm_result).tolist().index(max(mm_result)), max(mm_result)])
  np.array(mm_result[np.array(mm_result).tolist().index(max(mm_result))]) = -1


#for i in range(len(mm_result)):
#  #max 뽑고, index도 받아서 append
#  if(max(mm_result)):
#    max = max(mm_result)
#    index = indexOf

SyntaxError: ignored

In [ ]:
max(mm_result).output

In [ ]:
mm_result[np.array(mm_result).tolist().index(max(mm_result))][0]#여기서 57787을 -1로 바꿔주면 좋을듯.

In [ ]:
sorted_arr

In [ ]:
#같은 값이 2개 이상인 경우 1,2만 비교
#적합도는 어떻게 판별하지? ex) 정확도 60%면 추가질문?

In [ ]:
y_prob = model.predict(pat_padded, verbose=1) 
predicted = y_prob.argmax(axis=-1)

In [ ]:
# predict = model.predict(test_padded, verbose=0)
# print ('Predict:\n',  predict)

# # evalue
# loss_and_metric = model.evaluate(test_padded, test_labels)
# print ('Evaluate:\n', loss_and_metric)

In [ ]:
print("결과 출력")

encoder.classes_[predicted]
#산후 우울증	우울,초조함,불안

In [ ]:
print(train['disease'].tolist().index('의상도착증'))
print(mm_result[49])